In [1]:
##### 필요한 패키지를 로드합니다.
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import *
from PIL import ImageTk, Image
import glob
import tensorflow as tf
from keras.models import load_model
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib import pyplot as plt
import os


##### 변수 설정
LABELS = ['recycle', 'trash']  # 여러분들 사진의 라벨로 변경해주세요
CONFIDENCE = 0.3
THRESHOLD = 0.3  # NMS(Num Max Suppression)

# 코랩에서 내려받은 환경 구성 파일과 가중치로 셋팅합니다.
net = cv2.dnn.readNetFromDarknet('C:\\data\\yolov4_custom.cfg','C:\\data\\yolov4_custom_final.weights')

test_list = []   # 테스트 파일 리스트
test_num = 0     # 현재 테스트 파일 번호
graph_flag = False  # 그래프 존재 여부


##### 이미지를 분류하는 함수
def classify():
    if not test_list:  # 선택한 파일이 없다면
        tk.messagebox.showwarning("", "파일을 선택해주세요.")

    else:
        img = cv2.imread(test_list[test_num])  # 이미지를 숫자로 변환
        resize_img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)  # 32 x 32 로 reshape                    

        new_model = load_model('c:\\data\\recycle_model.h5')  # 학습 시킨 모델 불러오기

        x4 = tf.reshape(resize_img, (1,32,32,3))
        x4 = x4/255  # 정규화

        results = new_model.predict(x4)
        a = np.argmax(results)

        target_dict = {
         0: '분리수거 가능',
         1: '분리수거 불가능'
        }
        
        label_result.configure(text=target_dict[a], borderwidth=2, relief="groove")
        label_result.pack()
        label_result.place(relx=0.4, rely=0.7)
        
        draw_graph(results)
        
        
##### 확률 그래프 그리는 함수
def draw_graph(pred):
    global plot
    global graph_flag
    
    if not graph_flag:
        fig = plt.Figure(figsize=(5, 0.8), facecolor='#ebfeff')  # 그래프의 크기와 색깔을 지정
        plot = FigureCanvasTkAgg(fig, master=top)  # tkinter 위에 그래프 올리기
        
        y = ['recycle %0.3f'%pred[0][0], 'trash %0.3f'%pred[0][1]]  # 소수점이하 3번째까지만 2개의 확률 저장
        fig.add_subplot(1,1,1).barh(y, pred[0], color='#a7e4e7')  # 그래프 그리기, 막대 색깔 설정
        fig.tight_layout()
        
        plot.draw()                  # 그래프 그리기
        plot.get_tk_widget().pack()  # 그래프 설정을 저장하기
        plot.get_tk_widget().place(relx=0.15, rely=0.76)  # 그래프 위치 지정하기
        
        graph_flag = True

    
##### 원본 이미지를 넣으면 사물 검출해주는 함수
def detect():
    if not test_list:  # 선택한 파일이 없다면
        tk.messagebox.showwarning("", "파일을 선택해주세요.")

    else:
        img = cv2.imread(test_list[test_num])
        resize_img = cv2.resize(img, (500, 350))
        H, W, _ = resize_img.shape
        blob = cv2.dnn.blobFromImage(resize_img, scalefactor=1/255., size=(416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        output = net.forward()

        boxes, confidences, class_ids = [], [], []

        for det in output: # output [:4]:x,y,w,h, [5:]:score
            box = det[:4]
            scores = det[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > CONFIDENCE:
                cx, cy, w, h = box * np.array([W, H, W, H])  # 역정규화함
                x = cx - (w / 2)  # 중심찾는것
                y = cy - (h / 2)  # 중심찾는것

                boxes.append([int(x), int(y), int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

        # Num Max Suppression # https://naknaklee.github.io/etc/2021/03/08/NMS/
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, THRESHOLD)

        if len(idxs) > 0:
            cnt = 0
            for i in idxs.flatten():        
                x, y, w, h = boxes[i]
                pt1=(x, y)
                pt2=(x + w, y + h)
                cnt += 1

                # 결과 상자, 텍스트 색상 설정
                if class_ids[i] == 0:
                    color = (29, 219, 22)
                else:
                    color = (0, 0, 255)

                # 사물 검출 결과 상자 생성
                cv2.rectangle(resize_img, pt1=(x, y), pt2=(x + w, y + h), color=color, thickness=2)

                # 사물 검출 결과 텍스트 생성
                if x <= 10:
                    x = 10
                if y <= 10:
                    x = boxes[i][0] + w - 110
                    y = boxes[i][1] + h + 10    
                    
                cv2.putText(resize_img, text='%s %.2f' % (LABELS[class_ids[i]], confidences[i]), org=(x, y), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=color, thickness=10)
                cv2.putText(resize_img, text='%s %.2f' % (LABELS[class_ids[i]], confidences[i]), org=(x, y), 
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(25, 25, 25), thickness=2)
        
        # 사물 검출된 결과 이미지가 생성되는 위치와 이름을 확인하세요.
        file_name = os.path.splitext(os.path.basename(test_list[test_num]))
        cv2.imwrite('c:\\data\\img\\detect\\detection_' + str(file_name[0]) + '.png', resize_img)
        detect_image()
        
    
##### 이미지를 업로드하는 함수 생성
def upload_image():
    global test_num

    try:
        test_num = 0 
        plt.clf() # 현재 그림을 지웁니다. 
        
        global test_list  # 변수를 global 변수로 생성합니다. 
                          # 다른 함수에서도 이 변수를 사용할 수 있습니다.
            
        test_list = filedialog.askopenfilenames()  # 윈도우 탐색기를 열고 파일을 선택하는 창
                                                   # 그 파일이 있는 위치와 이름이 test_list 에 담김
        Next('first')
       
    except:
        pass  # 여기에 pass 가 있기 때문에 이미지가 아닌 다른 csv 파일을 업로드 하게 되면
              # 그냥 아무것도 안하고 가만히 있습니다.

        
##### 사진을 부착하는 함수 생성
def draw_image():
    if not test_list:
        uploaded = Image.open('c:\\data\\img\\main.png')
    else:
        uploaded = Image.open(test_list[test_num])  # 이미지를 불러와서 uploaded 라는 변수에 넣고
        
    uploaded = uploaded.resize((500, 350))  # 업로드 되는 이미지의 사이즈를 
                                            # 화면에 보이기 적당하도록 변경합니다.

    im = ImageTk.PhotoImage(uploaded)  # 이미지를 tkinter 용 이미지로 변경하는 작업
    sign_image.configure(image=im)  # 이미지를 tkinter 창에 출력하기 위한 작업
                                    # sign_image 는 앞에서 먼저 확보한 영역입니다.
    sign_image.image = im   # 그 영역에 업로드한 이미지를 붙여라


##### 사물 검출한 사진을 부착하는 함수 생성
def detect_image():
    plt.clf()  # 현재 그림을 지웁니다. 
    try:
        file_name = os.path.splitext(os.path.basename(test_list[test_num]))
        file_path = 'C:\\data\\img\\detect\\detection_' + str(file_name[0]) + '.png'  # 사물검출한 결과를 file_path 에 담습니다.   
                                                                              # 이 위치는 detect 함수의 위치와 같아야 합니다,
        uploaded2 = Image.open(file_path)   # 이미지를 uploaded2 라는 변수에 담고
        uploaded2 = uploaded2.resize((500, 350))  # 사이즈 조절하고 

        im2 = ImageTk.PhotoImage(uploaded2)  # 이미지를 tkinter 창에 출력
        sign_image.configure(image=im2)
        sign_image.image = im2

    except:
        pass  # try ~ except 사이의 코드에 데이터 상의 오류가 생겨도 그냥 가만히 있어라

    
##### 이전/다음 파일 가져오는 함수
def Next(flag):
    global test_num
    global graph_flag
    fin = False

    if not test_list and flag != 'first':  # 선택한 파일이 없다면
        tk.messagebox.showwarning("", "파일을 선택해주세요.")
        
    else:
        if flag == 'plus':  # 다음 버튼
            test_num += 1
            if test_num >= len(test_list):
                tk.messagebox.showwarning("", "마지막 파일입니다.")
                test_num -= 1
                fin = True

        elif flag == 'minus':  # 이전 버튼
            test_num -= 1
            if test_num < 0:
                tk.messagebox.showwarning("", "첫번째 파일입니다.")
                test_num += 1
                fin = True

        if not fin:  # 이전/다음 버튼으로 파일을 변경했을 때
            label_result.place_forget()
            draw_image()
            if graph_flag:  # 확률 그래프가 그려져 있다면
                plot.get_tk_widget().destroy()
                graph_flag = False
        
    
##### 화면 생성
top = tk.Tk()                          # tkinter 클래스로 top 이라는 객체를 만듭니다. 
top.geometry("700x600+400+150")        # 창 크기 및 화면에서의 위치
top.resizable(False, False)           # 창 크기 변경 막기
top.title('Recycle Image Classification')    # 창위에 제목을 답니다. 
top.configure(background='#ebfeff')      # 창 백그라운드 색깔

img = Image.open('c:\\data\\img\\main.png')  # 창에 기본적으로 표시할 기본 사진
img = img.resize((500, 350))                 # 원본 올린 사진의 사이즈 변경
im = ImageTk.PhotoImage(img, master=top)     # 사진을 창에 올리기위해 im 변수에 담습니다.
                                             # tkinter 용 이미지로 변경합니다.
sign_image = Label(top, image=im)            # 사진을 화면에 붙이기 위한 sing_image 를 생성
sign_image.pack(side=BOTTOM, expand=True)   # 사진(sing_image) 을 tkinter 창에 packing(붙임) 합니다.
                                             # expand=True 요구되지 않은 공간활용하기 
sign_image.place(relx=0.15, rely=0.15)      # x 좌표의 배치 비율, y 좌표의 배치 비율 
    
    
##### 라밸 생성
# 화면 타이틀
heading = Label(top, text="페트병 라벨 유무 분류 인공지능", pady=27, font=('CG Omega',20,'bold'))
heading.configure(background='#ebfeff', foreground='#81d6db')  # 글씨 색깔 변경
heading.pack()  # 글씨를 화면에 표시합니다.

# 이미지 분류 결과 글씨
label_result = Label(top, text="", pady=0, font=('CG Omega',18,'bold'))
label_result.configure(background='#ebfeff', foreground='black')  # 글씨 색깔 변경


##### 버튼 생성
# upload 버튼 생성
btn_upload = Button(top, text="Upload", relief=RIDGE, command=upload_image, padx=10, pady=5)
# Button 함수로 upload 버튼을 만드는데 top (화면 캔버스)에 버튼을 만듭니다.
# command 에 버튼의 기능을 구현하는데 버튼을 눌렀을 때 작동하는 함수a가 upload_image 입니다.
# 버튼의 크기를 padx 와 pady 로 조정합니다.
btn_upload.place(relx=0.3, rely=0.9)  # 버튼의 위치를 지정합니다.
btn_upload.configure(background='#a7e4e7', foreground='white', font=('arial',12,'bold'))  # 버튼의 색깔과 폰트 

# classify 버튼 생성
btn_classify = Button(top,text="Classify", relief=RIDGE, command=classify, padx=10, pady=5)
btn_classify.configure(background='#a7e4e7', foreground='white', font=('arial',12,'bold'))
btn_classify.place(relx=0.45, rely=0.9)

# object detection 버튼 생성
btn_detect = Button(top, text="Detect", relief=RIDGE, command=detect, padx=10, pady=5)
# relief 는 버튼 스타일, detect 버튼은 detect_image 함수를 호출하는 기능을 합니다.
btn_detect.configure(background='#a7e4e7', foreground='white', font=('arial',12,'bold'))
btn_detect.place(relx=0.6, rely=0.9)  # detect 버튼의 위치를 지정

# 이전 버튼 생성
img = Image.open('C:\\data\\img\\left_arrow.png')
img = img.resize((30, 30))
img_before = ImageTk.PhotoImage(img, master=top)
btn_before = Button(top, image=img_before, background='#ebfeff', width=30, height=30, borderwidth=0, font=('Courier', 10), command=lambda:Next('minus'))
btn_before.place(relx=0.09, rely=0.68)

# 다음 버튼 생성
img = Image.open('C:\\data\\img\\right_arrow.png')
img = img.resize((30, 30))
img_next = ImageTk.PhotoImage(img, master=top)
btn_next = Button(top, image=img_next, background='#ebfeff', width=30, height=30, borderwidth=0, font=('Courier', 10), command=lambda:Next('plus'))
btn_next.place(relx=0.88, rely=0.68)


top.mainloop()  # 계속 창을 띄우려면 루프문을 돌려야해서 필요합니다.

1/1 [==============================] - 0s 252ms/step


<Figure size 640x480 with 0 Axes>